In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import boto3
from io import StringIO # python3; python2: BytesIO 
#url='https://www.toronto.ca/data/children/dmc/a2z/a2za.html'
#df.append(pd.read_html(url))
#this loads all the tables in the page

In [ ]:
import numpy as np

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
website='https://www.toronto.ca/data/children/dmc/a2z/a2za.html'
result=requests.get(website)
content=result.text
soup=BeautifulSoup(content,'lxml')


**Retrieving all the urls's for A to z listing**

In [ ]:
main='https://www.toronto.ca/data/children/dmc/a2z/'
list_of_pages=soup.find(class_='phone')
alpha_list=[listing['href'] for listing in list_of_pages.find_all('a', href=True)]
#alpha_list

**1.Extracting Subsidy information for all the childcare centres**

In [ ]:
data=[]
for url in alpha_list:
    website1=main+url
    result1=requests.get(website1)
    content1=result1.text
    soup1=BeautifulSoup(content1,'lxml')
    table=soup1.findAll('table')[0]
    header=table.find_all('th')
    rows=table.find_all('tr')
    for row in rows[1:]:
        data.append(row.getText(strip=True,separator='\n'))


Writing the subsidy information to a DataFrame

In [ ]:
cols=['Centre_Name','Fee_subsidy_accepted']
table_subsidy_info=pd.DataFrame(columns=cols) 
for i in range(len(data)):
    table_subsidy_info.loc[i,table_subsidy_info.columns[0]]=data[i].split('\n')[0]
    table_subsidy_info.loc[i,table_subsidy_info.columns[1]]=data[i].split('\n')[1]


**2.Extracting Centre information**

Store the links of all the centres

In [ ]:
main='https://www.toronto.ca/data/children/dmc/a2z/'
root = 'https://www.toronto.ca/data/children/dmc'
links=[]
for mlink in alpha_list:
    concat=main+str(mlink)
    result2=requests.get(concat)
    content2=result2.text
    soup2 = BeautifulSoup(content2, 'lxml')
    tabular2=soup2.find(class_='pfrPrdListing')
    for link in tabular2.find_all('a', href=True):
        links.append(root+str(link['href']).replace('..',''))


Extract data from each url

In [ ]:
centre=[]
Address=[]
Ward=[]
Age=[]
Contact=[]
for link in links:
    result=requests.get(link)
    content=result.text
    soup=BeautifulSoup(content,'lxml')
    centre.append(soup.find(class_="active").get_text())
    Address.append(soup.find('div',class_="csd_opcrit_content_box").find('p').get_text(strip=True,separator=','))
    Ward.append(soup.find('div',class_="csd_opcrit_content_box").find('span').get_text(strip=True) )
    Age.append(soup.find('div',class_="csd_opcrit_content_box").find('strong').get_text(strip=True) if soup.find('div',class_="csd_opcrit_content_box").find('strong') else '')
    Contact.append(soup.find(class_="nudge").get_text(strip=True) if soup.find(class_="nudge") else '')
    #if soup.find('div',class_="csd_opcrit_content_box").find('strong'):
        #Age.append(soup.find('div',class_="csd_opcrit_content_box").find('strong').get_text(strip=True))
    #else:
        #Age.append("")

Dataframe for Centre contact information

In [ ]:
dic={'Centre_name':centre,'Address':Address,'Ward':Ward,'Age':Age,'Contact':Contact}
Centre_information=pd.DataFrame(dic)
Centre_information

**3.Early years program information**

In [ ]:
cols=['Program_name','Capacity','Vacancy','Quality_Rating','Center_name','Before_after_school_program','Center_address']
Early_years_program_info=pd.DataFrame(columns=cols) 
for link in links:
    table_program_info=pd.DataFrame(columns=cols)
    data=[]
    result1=requests.get(link)
    content1=result1.text
    soup1=BeautifulSoup(content1,'lxml')
    table_all=soup1.findAll('table')
    cc_name=soup1.find(class_="active").get_text()
    cc_address=soup1.find('div',class_="csd_opcrit_content_box").find('p').get_text(strip=True,separator=',')
    #print(link,len(table_all))
    #print(cc_address)
    #print(link,len(table_all))
    if len(table_all)==0:
        continue;
    else:
        table=soup1.findAll('table')[0]
        header=table.find_all('th')
        #cols=[]
        #for th in header:
            #cols.append(th.getText())
        #cols.append('Center_name')
        rows=table.find_all('tr')
        for row in rows[1:]:
            data.append(row.getText(strip=True,separator=',').replace(',/','/'))
        data.append(cc_name)
        for i in range(len(data)-1):
            table_program_info.loc[i,table_program_info.columns[0]]=data[i].split(',')[0]
            table_program_info.loc[i,table_program_info.columns[1]]=data[i].split(',')[1]
            table_program_info.loc[i,table_program_info.columns[2]]=data[i].split(',')[2]
            table_program_info.loc[i,table_program_info.columns[3]]=data[i].split(',')[3]
            table_program_info.loc[i,table_program_info.columns[4]]=data[-1]
            table_program_info.loc[i,table_program_info.columns[5]]='No'
            table_program_info.loc[i,table_program_info.columns[6]]=cc_address
        Early_years_program_info=Early_years_program_info.append(table_program_info,ignore_index=True)
    

**4.Before_after_school_program_information**

In [ ]:
cols=['Program_name','Capacity','Vacancy','Quality_Rating','Center_name','Before_after_school_program','Center_address']
Before_after_school_program_info=pd.DataFrame(columns=cols) 
for link in links:
    table_program_info=pd.DataFrame(columns=cols)
    data=[]
    result1=requests.get(link)
    content1=result1.text
    soup1=BeautifulSoup(content1,'lxml')
    table_all=soup1.findAll('table')
    cc_name=soup1.find(class_="active").get_text()
    cc_address=soup1.find('div',class_="csd_opcrit_content_box").find('p').get_text(strip=True,separator=',')
    #print(link,len(table_all))
    if len(table_all)<=1:
        continue;
    else:
        table=soup1.findAll('table')[1]
        header=table.find_all('th')
        #cols=[]
        #for th in header:
            #cols.append(th.getText())
        #cols.append('Center_name')
        rows=table.find_all('tr')
        for row in rows[1:]:
            data.append(row.getText(strip=True,separator=',').replace(',/','/'))
        data.append(cc_name)
        for i in range(len(data)-1):
            table_program_info.loc[i,table_program_info.columns[0]]=data[i].split(',')[0]
            table_program_info.loc[i,table_program_info.columns[1]]=data[i].split(',')[1]
            table_program_info.loc[i,table_program_info.columns[2]]=data[i].split(',')[2]
            table_program_info.loc[i,table_program_info.columns[3]]=data[i].split(',')[3]
            table_program_info.loc[i,table_program_info.columns[4]]=data[-1]
            table_program_info.loc[i,table_program_info.columns[5]]='Yes'
            table_program_info.loc[i,table_program_info.columns[6]]=cc_address
        Before_after_school_program_info=Before_after_school_program_info.append(table_program_info,ignore_index=True)
#table_after_program_info_final

**Data Cleansing**

1.Missing Values

In [ ]:
table_subsidy_info.isnull().sum()

In [ ]:
Centre_information.isnull().sum()


In [ ]:
Early_years_program_info.isnull().sum()

In [ ]:
Before_after_school_program_info.isnull().sum()

2.Removing Unnecessary information

In [ ]:
Centre_information.describe()

In [ ]:
Centre_information['Age'].value_counts()

In [ ]:
Centre_information['Age']=Centre_information['Age'].str.replace('serves children',"")

In [ ]:
Early_years_program_info.describe()

In [ ]:
Before_after_school_program_info.describe()

3.Duplicate Check

In [ ]:
table_subsidy_info[table_subsidy_info.duplicated()]

Drop the records as they are exact duplicates

In [ ]:
l=table_subsidy_info[table_subsidy_info.duplicated()].index
table_subsidy_info=table_subsidy_info.drop_duplicates()


In [ ]:
Centre_information[Centre_information.duplicated()]

In [ ]:
table_program_info_final[table_program_info_final.duplicated()]


In [ ]:
table_after_program_info_final[table_after_program_info_final.duplicated()]

4.Inconsistent Data

In [ ]:
table_subsidy_info['Fee_subsidy_accepted'].unique()

In [ ]:
table_subsidy_info[table_subsidy_info['Fee_subsidy_accepted']=='coming soon']

In [ ]:
table_subsidy_info['Fee_subsidy_accepted']=table_subsidy_info['Fee_subsidy_accepted'].str.replace('coming soon','No')

In [ ]:
Centre_information.describe()

The Count of all the fields are not same which shows that there are some inconsistencies

In [ ]:
Centre_information[Centre_information['Age']=='']


Records with blank values are extended day programs hence they can be replaced with the value"Kindergarten to Grade 5/6"

In [ ]:
l=Centre_information[Centre_information['Age']==''].index
for i in l:
    Centre_information.loc[i,'Age']='Kindergarten to Grade 5/6'
Centre_information[Centre_information['Age']=='']

In [ ]:
Centre_information[Centre_information['Contact']=='']

The null values for Contact are updated as 'No contact information'

In [ ]:
l=Centre_information[Centre_information['Contact']==''].index
for i in l:
    Centre_information.loc[i,'Contact']='No Contact Information'
Centre_information[Centre_information['Contact']=='']

In [ ]:
Removal of \n\t

In [ ]:
Centre_information['Contact']=Centre_information['Contact'].str.replace('\t','')
Centre_information['Address']=Centre_information['Address'].str.replace('\t','')
Centre_information['Address']=Centre_information['Address'].str.replace('\n','')

In [ ]:
Early_years_program_info['Center_address']=Early_years_program_info['Center_address'].str.replace('\t','')
Early_years_program_info['Center_address']=Early_years_program_info['Center_address'].str.replace('\n','')
Before_after_school_program_info['Center_address']=Before_after_school_program_info['Center_address'].str.replace('\t','')
Before_after_school_program_info['Center_address']=Before_after_school_program_info['Center_address'].str.replace('\n','')

In [ ]:
Early_years_program_info['Quality_Rating']=Early_years_program_info['Quality_Rating'].str.replace('-','')
Before_after_school_program_info['Quality_Rating']=Before_after_school_program_info['Quality_Rating'].str.replace('-','')
Early_years_program_info['Vacancy']=Early_years_program_info['Vacancy'].str.replace('Unknown','Not updated')
Before_after_school_program_info['Vacancy']=Before_after_school_program_info['Vacancy'].str.replace('Unknown','Not updated')

Writing the Dataframes as csv file to AWS s3

In [ ]:
session = boto3.Session( 
         aws_access_key_id='<id>', 
         aws_secret_access_key='<key>')

#Then use the session to get the resource
s3 = session.resource('s3')
my_bucket = s3.Bucket('<bucket>')
csv_buffer=StringIO()
centre_csv_buffer=StringIO()
table_csv_buffer=StringIO()
after_csv_buffer=StringIO()
table_subsidy_info.to_csv(csv_buffer)
Centre_information.to_csv(centre_csv_buffer)
Early_years_program_info.to_csv(table_csv_buffer)
Before_after_school_program_info.to_csv(after_csv_buffer)
#dynamodb=session.resource('dynamodb',region_name='ca-central-1')
s3.Object('<bucket>','Project/Subsidy_fee/Subsidy_fee.csv').put(Body=csv_buffer.getvalue())
s3.Object('<bucket>','Project/Centre_information/Centre_information.csv').put(Body=centre_csv_buffer.getvalue())
s3.Object('<bucket>','Project/Early_Years_program_information/Early_Years_program_information.csv').put(Body=table_csv_buffer.getvalue())
s3.Object('<bucket>','Project/Before_after_school_program_information/Before_after_school_program_information.csv').put(Body=after_csv_buffer.getvalue())